In [ ]:
import numpy as np

In [ ]:
# scalar shrinkage operator
# returns sgn(x) max(|x| - eps, 0)
def ScalarShrinkage(x, eps):
    return x[abs(x) < eps] = 0.0

# returns U ScalarShrinkage(sigma, eps) V^T
def S(x, eps):
    U, s, V = np.linalg.svd(x)
    return (U.dot(ScalarShrinkage(s, eps))).dot(V.T)

# need to process x
def P(x, omega):
    return x

In [ ]:
def compute_optproblem(B1, B2, E, W, l, alpha, omega, N_ITERS = 10, nu1 = 3, nu2 = 3, p = 0.7):
    def update_A(Ak, Wk, Ek, Y1k, Y2k, uk):
        return S(1 / uk, Wk - Y1k / uk)
    
    def update_W(Ak, Wk, Ek, Y1k, Y2k, uk):
        PWk = P(B1.dot(W).dot(B2.T) + Ek - D + Y2k / uk, omega)
        x = Wk - ((B1.T.dot(PWk)).dot(B2) + Wk - Ak - Y1k / uk) / nu1
        return ScalarShrinkage(x, l / (uk * nu1))
    
    def update_E(Ak, Wk, Ek, Y1k, Y2k, uk):
        PEk = P(E + (B1.dot(Wk)).dot(B2.T) - D, omega)
        x = Ek - (PEk  + Y2k / uk) / nu2
        return ScalarShrinkage(x, alpha / (uk * nu2))
    
    def update_Y1(Ak, Wk, Ek, Y1k, Y2k, uk):        
        return Y1k + uk * P((B1.dot(Wk)).dot(B2.T) + Ek - D, omega)
    
    def update_Y2(Ak, Wk, Ek, Y1k, Y2k, uk):
        return Y2k + uk * (Ak - Wk)
    
    Ak = W.copy()
    Wk = W.copy()
    Ek = E.copy()
    Y1k = 0
    Y2k = 0
    # what is u0?
    uk = 1
    
    Y1k = update_Y1(Ak, Wk, Ek, Y1k, Y2k, uk, l)
    Y2k = update_Y2(Ak, Wk, Ek, Y1k, Y2k, uk, l)
    
    for i in range(N_ITERS):
        Ak = update_A(Ak, Wk, Ek, Y1k, Y2k, uk, l)
        Wk = update_W(Ak, Wk, Ek, Y1k, Y2k, uk, l)
        Ek = update_E(Ak, Wk, Ek, Y1k, Y2k, uk, l)
        Y1k = update_Y1(Ak, Wk, Ek, Y1k, Y2k, uk, l)
        Y2k = update_Y2(Ak, Wk, Ek, Y1k, Y2k, uk, l)
        uk *= p
        
    return Wk, Ek

In [ ]:
# update omega
def supp(E):
    return omega

def comute_all(B1, B2, E, W, l, alpha, omega, N_ITERS = 10):    
    Wk = W
    Ek = E
    
    # what is omega0
    omega_k = set()
    for i in range(N_ITERS):
        Wk, Ek = compute_optproblem(B1, B2, E, W, l, alpha, omega_k)
        omega_k = omega_k.difference(supp(Ek))
        
    return Wk